In [1]:
import streamlit as st
from AuxSearch.components.PDFTextExtractor import PDFConverter
from AuxSearch.components.TextChunkerEmbedder import TextProcessor
from AuxSearch.components.Model import ChainLoader
# from AuxSearch.components.StreamlitUserInput import UserInputHandler

pdfconverter = PDFConverter()
textprocessor = TextProcessor()
chains = ChainLoader()


In [2]:
import streamlit as st
from AuxSearch.components.PDFTextExtractor import PDFConverter
from AuxSearch.components.TextChunkerEmbedder import TextProcessor
from AuxSearch.components.Model import ChainLoader
from langchain.embeddings import OllamaEmbeddings 
from langchain_community.vectorstores.faiss import FAISS
from AuxSearch.constants import *

pdfconverter = PDFConverter()
textprocessor = TextProcessor()
chains = ChainLoader()


def user_input(user_question):
    embeddings = OllamaEmbeddings()
    
    new_db = FAISS.load_local("faiss_index", embeddings)
    docs = new_db.similarity_search(user_question)

    chain = chains.get_chains()

    
    response = chain(
        {"input_documents":docs, "question": user_question}
        , return_only_outputs=True)

    print(response)
    st.write("Reply: ", response["output_text"])


In [3]:
embeddings = OllamaEmbeddings()

vectorstore = FAISS.load_local("faiss_index",embeddings)

In [4]:
retriever = vectorstore.as_retriever(search_type = "similarity",k=3)

In [5]:
user_question = "What is the name on resume?"
pdf_files = "Omkar_Firame_Resume.pdf"



In [6]:
raw_text = pdfconverter.pdf_to_text(pdf_files)


In [7]:

text_chunks = textprocessor.get_chunks(raw_text)


In [8]:
text_chunks

["Omkar\nFirame\n+919404350615\n|\nE-Mail\n|\nLinkedin\n|\nGitHub\nSUMMAR Y:\nData\nScientist\nwith\nstrong\nMachine\nLearning\nfoundation\nand\n3+\nyears\nof\nexperience\nin\ndeveloping\nrobust\nML\nmodels\nusing\npredictive\ndata\nmodeling,\nanalyze\ndata\nto\ndeliver\ninsights\nand\nimplement\naction-oriented\nsolutions\nto\ncomplex\nbusiness\nproblems.\nTECHNICAL\nSKILLS:\nPython,\nR,\nMachine\nLearning,\nDeep\nLearning,\nPyTorch,\nIguazio,\nKubeFlow\nPipeline,\nAzure\nDatabricks,\nDask,\nWeb\nScraping\n(Selenium),\nGit,\nEXPERIENCE:\nData\nScientist\n-\nWynum\nAutomation\nServices\nPvt.\nLtd\n|\nApril\n2023\n-\nPresent\n●\nManaged\na\nproject\nimplementing\nan\ninformation\nretrieval\nsystem\nfor\nextracting\nrelevant\npress\nreleases\nfrom\nonline\nsources.\n●\nApplied\nadvanced\ndata\npreprocessing\ntechniques\nto\ncleanse\nand\nstructure\ncontent,\nenhancing\ncategorization\nand\nanalytical\ncapabilities.\n●\nDeveloped\ncustom\nweb\nscrapers\nusing\nPython\n(Beautiful\nSoup,\nR

In [31]:

textprocessor.get_vector_store(text_chunks)

In [9]:
embeddings = OllamaEmbeddings()


In [33]:

new_db = FAISS.load_local("faiss_index", embeddings)


In [34]:
new_db

In [35]:

docs = new_db.similarity_search(user_question)


In [36]:
docs

[Document(page_content="Omkar\nFirame\n+919404350615\n|\nE-Mail\n|\nLinkedin\n|\nGitHub\nSUMMAR Y:\nData\nScientist\nwith\nstrong\nMachine\nLearning\nfoundation\nand\n3+\nyears\nof\nexperience\nin\ndeveloping\nrobust\nML\nmodels\nusing\npredictive\ndata\nmodeling,\nanalyze\ndata\nto\ndeliver\ninsights\nand\nimplement\naction-oriented\nsolutions\nto\ncomplex\nbusiness\nproblems.\nTECHNICAL\nSKILLS:\nPython,\nR,\nMachine\nLearning,\nDeep\nLearning,\nPyTorch,\nIguazio,\nKubeFlow\nPipeline,\nAzure\nDatabricks,\nDask,\nWeb\nScraping\n(Selenium),\nGit,\nEXPERIENCE:\nData\nScientist\n-\nWynum\nAutomation\nServices\nPvt.\nLtd\n|\nApril\n2023\n-\nPresent\n●\nManaged\na\nproject\nimplementing\nan\ninformation\nretrieval\nsystem\nfor\nextracting\nrelevant\npress\nreleases\nfrom\nonline\nsources.\n●\nApplied\nadvanced\ndata\npreprocessing\ntechniques\nto\ncleanse\nand\nstructure\ncontent,\nenhancing\ncategorization\nand\nanalytical\ncapabilities.\n●\nDeveloped\ncustom\nweb\nscrapers\nusing\nPython

In [10]:
# prompt_template = """
#     Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
#     provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
#     Context:\n {context}?\n
#     Question: \n{question}\n

#     Answer:
# """

In [ ]:
prompt_template = """
    Answer the question to the point without description. If answer not found in context reply "Answer Not found"\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
"""

In [11]:
from langchain.llms import CTransformers


In [12]:
MODEL_PATH

WindowsPath('C:/Users/omkar/Downloads/LLama2-Model/llama-2-7b-chat.ggmlv3.q8_0.bin')

In [13]:
from langchain.prompts import PromptTemplate

model = CTransformers(model = 'C:/Users/omkar/Downloads/LLama2-Model/llama-2-7b-chat.ggmlv3.q8_0.bin',model_type='llama')
prompt = PromptTemplate(template=prompt_template,input_variables=["context","question"])


c:\Users\Omkar\Anaconda3\envs\pyenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
prompt

PromptTemplate(input_variables=['context', 'question'], template='\n    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in\n    provided context just say, "answer is not available in the context", don\'t provide the wrong answer\n\n\n    Context:\n {context}?\n\n    Question: \n{question}\n\n\n    Answer:\n')

In [15]:
from langchain_core.runnables import RunnablePassthrough

In [16]:
from langchain.schema import StrOutputParser

In [17]:
rag_chain = (
    {
        "context":retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser
)

In [18]:
rag_chain.invoke(user_question)

Number of tokens (1603) exceeded maximum context length (512).


In [19]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(model,chain_type="stuff",prompt=prompt)

In [24]:
docs[0].page_content

"Omkar\nFirame\n+919404350615\n|\nE-Mail\n|\nLinkedin\n|\nGitHub\nSUMMAR Y:\nData\nScientist\nwith\nstrong\nMachine\nLearning\nfoundation\nand\n3+\nyears\nof\nexperience\nin\ndeveloping\nrobust\nML\nmodels\nusing\npredictive\ndata\nmodeling,\nanalyze\ndata\nto\ndeliver\ninsights\nand\nimplement\naction-oriented\nsolutions\nto\ncomplex\nbusiness\nproblems.\nTECHNICAL\nSKILLS:\nPython,\nR,\nMachine\nLearning,\nDeep\nLearning,\nPyTorch,\nIguazio,\nKubeFlow\nPipeline,\nAzure\nDatabricks,\nDask,\nWeb\nScraping\n(Selenium),\nGit,\nEXPERIENCE:\nData\nScientist\n-\nWynum\nAutomation\nServices\nPvt.\nLtd\n|\nApril\n2023\n-\nPresent\n●\nManaged\na\nproject\nimplementing\nan\ninformation\nretrieval\nsystem\nfor\nextracting\nrelevant\npress\nreleases\nfrom\nonline\nsources.\n●\nApplied\nadvanced\ndata\npreprocessing\ntechniques\nto\ncleanse\nand\nstructure\ncontent,\nenhancing\ncategorization\nand\nanalytical\ncapabilities.\n●\nDeveloped\ncustom\nweb\nscrapers\nusing\nPython\n(Beautiful\nSoup,\nRe

In [22]:
user_question

'What is the name on resume?'

In [20]:
response = chain(
    {"input_documents":docs, "question": user_question}
    , return_only_outputs=True)

print(response)

c:\Users\Omkar\Anaconda3\envs\pyenv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Number of tokens (1301) exceeded maximum context length (512).
Number of tokens (1302) exceeded maximum context length (512).
Number of tokens (1303) exceeded maximum context length (512).
Number of tokens (1304) exceeded maximum context length (512).
Number of tokens (1305) exceeded maximum context length (512).
Number of tokens (1306) exceeded maximum context length (512).
Number of tokens (1307) exceeded maximum context length (512).
Number of tokens (1308) exceeded maximum context length (512).
Number of tokens (1309) exceeded maximum context length (512).
Number of tokens (1310) exceeded maximum context length (512).
Number of tokens (1311) exceeded maximum context length (512).
Number of tokens (1312) exceeded maximum context length (

{'output_text': 'The answer is not available in the information retrieval is not available in the answer is not available in the provided context is not available in the details are not available in the answer is not available in the answer is not available in the details about the detailed. Omkarthick\n Omkarth\nOmkarthickbox ____________  There is not available in the details on basis, No,   Please refer to provide all of course\n    Available\nNot available in the name is not available in the provided context does not available\nThere are not available in the answer is not available in the information retrieval\nThe answer is not available in the answer is not available in the answer is not available in the details are not available in the data scientist none of Omkarthesection:\n Omkarthick box link forOmkarthese,  The answer is not available in the name is not available in the answer is not available in the provided context does not available in the answer is not available in the 